In [1]:
from __future__ import print_function
import cv2 as cv
import argparse
import easydict
import time

In [2]:
def detectAndDisplay(frame):
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame_gray = cv.equalizeHist(frame_gray)

    #-- Detect faces
    faces = face_cascade.detectMultiScale(frame_gray)
    count = 0
    for (x,y,w,h) in faces:
        crop_img = frame[y:y+h, x:x+w]
        cv.imwrite("./images/frame%d.jpg" % round(time.time()), crop_img)     # save frame as JPEG file   
        count += 1
        center = (x + w//2, y + h//2)
        frame = cv.ellipse(frame, center, (w//2, h//2), 0, 0, 360, (255, 0, 255), 4)
        frame = cv.rectangle(frame, (x,y), (x+w,y+h), (255, 255, 0), 4)
        
        

        faceROI = frame_gray[y:y+h,x:x+w]
        #-- In each face, detect eyes
        eyes = eyes_cascade.detectMultiScale(faceROI)
        for (x2,y2,w2,h2) in eyes:
            eye_center = (x + x2 + w2//2, y + y2 + h2//2)
            radius = int(round((w2 + h2)*0.25))
            frame = cv.circle(frame, eye_center, radius, (255, 0, 0 ), 4)

    cv.imshow('Capture - Face detection', frame)

In [3]:
args = easydict.EasyDict({
        "face_cascade": 'data/haarcascade_frontalface_alt.xml',
        "eyes_cascade": 'data/haarcascade_eye_tree_eyeglasses.xml',
        "camera": 0
})

In [4]:
face_cascade_name = args.face_cascade
eyes_cascade_name = args.eyes_cascade
face_cascade = cv.CascadeClassifier()
eyes_cascade = cv.CascadeClassifier()

In [5]:
#-- 1. Load the cascades
if not face_cascade.load('/home/ai-lab1/.local/lib/python3.6/site-packages/cv2/'+face_cascade_name):
    print('--(!)Error loading face cascade')
    exit(0)
if not eyes_cascade.load('/home/ai-lab1/.local/lib/python3.6/site-packages/cv2/'+eyes_cascade_name):
    print('--(!)Error loading eyes cascade')
    exit(0)

In [6]:
camera_device = args.camera

In [7]:
#-- 2. Read the video stream
cap = cv.VideoCapture(camera_device)

In [8]:
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)

In [9]:
while True:
    ret, frame = cap.read()
    if frame is None:
        print('--(!) No captured frame -- Break!')
        break

    detectAndDisplay(frame)   
    if cv.waitKey(10) == 27:
        break